# Homework 6: Group 1
#### Logistic Regression

David Jung, Ryan Voges, Emily Blake, Spencer Powell, Abraham Alhomadi

In this exercise I want you to apply logistic regression model to the credit card data set which is available on the GitHub folder for HW6. This data set should be familiar as you have done an EDA on it in HW-3 EDA for Credit Card Default dataset on Kaggle.  Import the credit_card_clean.csv as a data frame and call it df. 

1. Change the type of the feature variables as you see fit! numerical variables vs categorical ones. You can use my answer key from HW3 as your reference. (5 points)


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
sns.set()  #if you want to use seaborn themes with matplotlib functions
import warnings
warnings.filterwarnings('ignore')

In [2]:
rand_state= 1000

In [3]:
df = pd.read_csv("credit_card_clean.csv")

In [4]:
df.head(10)

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,DEFAULT
0,1,20000.0,female,university,married,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,female,university,single,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,female,university,single,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,female,university,married,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,male,university,married,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0
5,6,50000.0,male,grad,single,37,0,0,0,0,...,19394.0,19619.0,20024.0,2500.0,1815.0,657.0,1000.0,1000.0,800.0,0
6,7,500000.0,male,grad,single,29,0,0,0,0,...,542653.0,483003.0,473944.0,55000.0,40000.0,38000.0,20239.0,13750.0,13770.0,0
7,8,100000.0,female,university,single,23,0,-1,-1,0,...,221.0,-159.0,567.0,380.0,601.0,0.0,581.0,1687.0,1542.0,0
8,9,140000.0,female,highschool,married,28,0,0,2,0,...,12211.0,11793.0,3719.0,3329.0,0.0,432.0,1000.0,1000.0,1000.0,0
9,10,20000.0,male,highschool,single,35,-2,-2,-2,-2,...,0.0,13007.0,13912.0,0.0,0.0,0.0,13007.0,1122.0,0.0,0


In [5]:
df.info()
# our data seems clean! No NA's

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID         30000 non-null  int64  
 1   LIMIT_BAL  30000 non-null  float64
 2   SEX        30000 non-null  object 
 3   EDUCATION  30000 non-null  object 
 4   MARRIAGE   30000 non-null  object 
 5   AGE        30000 non-null  int64  
 6   PAY_1      30000 non-null  int64  
 7   PAY_2      30000 non-null  int64  
 8   PAY_3      30000 non-null  int64  
 9   PAY_4      30000 non-null  int64  
 10  PAY_5      30000 non-null  int64  
 11  PAY_6      30000 non-null  int64  
 12  BILL_AMT1  30000 non-null  float64
 13  BILL_AMT2  30000 non-null  float64
 14  BILL_AMT3  30000 non-null  float64
 15  BILL_AMT4  30000 non-null  float64
 16  BILL_AMT5  30000 non-null  float64
 17  BILL_AMT6  30000 non-null  float64
 18  PAY_AMT1   30000 non-null  float64
 19  PAY_AMT2   30000 non-null  float64
 20  PAY_AM

In [6]:
for col in df.columns:
    print(col, df[col].nunique())

ID 30000
LIMIT_BAL 81
SEX 2
EDUCATION 4
MARRIAGE 3
AGE 56
PAY_1 11
PAY_2 11
PAY_3 11
PAY_4 11
PAY_5 10
PAY_6 10
BILL_AMT1 22723
BILL_AMT2 22346
BILL_AMT3 22026
BILL_AMT4 21548
BILL_AMT5 21010
BILL_AMT6 20604
PAY_AMT1 7943
PAY_AMT2 7899
PAY_AMT3 7518
PAY_AMT4 6937
PAY_AMT5 6897
PAY_AMT6 6939
DEFAULT 2


- SEX and DEFAULT should be dummy variables
- EDUCATION, PAY variables and MARRIAGE should be categorical! But why does MARRIAGE have three options?

In [7]:
# Dropped the 0 values from the MARRIAGE variable:
df = df[df.MARRIAGE != "other"]

In [8]:
categoricals = ["EDUCATION", "PAY_1", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6"]
dummies = ["SEX", "MARRIAGE", "DEFAULT"]

In [9]:
# Convert the floats and ints into categoricals
for col in categoricals:
    df[col]=df[col].astype("category")

2. Define your target variable. What are the proportions of default vs non-default in your data set? Is the target variable balanced or relatively imbalanced? 

3. Use get_dummies( drop_first=True ) function from pandas package to make the categorical variables into dummy variables. How many features you have now?

In [10]:
df = pd.get_dummies(df, columns = dummies, drop_first = True)

In [11]:
df.head(10)

,ID,LIMIT_BAL,EDUCATION,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,...,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,SEX_male,MARRIAGE_single,DEFAULT_1
0,1,20000.0,university,24,2,2,-1,-1,-2,-2,...,0.0,0.0,689.0,0.0,0.0,0.0,0.0,0,0,1
1,2,120000.0,university,26,-1,2,0,0,0,2,...,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,0,1,1
2,3,90000.0,university,34,0,0,0,0,0,0,...,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0,1,0
3,4,50000.0,university,37,0,0,0,0,0,0,...,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0,0,0
4,5,50000.0,university,57,-1,0,-1,0,0,0,...,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,1,0,0
5,6,50000.0,grad,37,0,0,0,0,0,0,...,20024.0,2500.0,1815.0,657.0,1000.0,1000.0,800.0,1,1,0
6,7,500000.0,grad,29,0,0,0,0,0,0,...,473944.0,55000.0,40000.0,38000.0,20239.0,13750.0,13770.0,1,1,0
7,8,100000.0,university,23,0,-1,-1,0,0,-1,...,567.0,380.0,601.0,0.0,581.0,1687.0,1542.0,0,1,0
8,9,140000.0,highschool,28,0,0,2,0,0,0,...,3719.0,3329.0,0.0,432.0,1000.0,1000.0,1000.0,0,0,0
9,10,20000.0,highschool,35,-2,-2,-2,-2,-1,-1,...,13912.0,0.0,0.0,0.0,13007.0,1122.0,0.0,1,1,0


In [12]:
# check the balance of the data for target variable 

4. Along with the target variable, define your feature space (X) and split the data into test (30%) and train set (70%)

5. From sklearn.linear_model import the relevant functions for Logistic Regression. Do the followings: (30 points)
    1. Train the logistic regression model using its default parameters. (5 points)
    2. Generate the predicted probabilities and predicted classifications and save them as y_hat_probs, y_hat respectively. (5 points)
    3. Plot the histogram of y_hat_probs? Explain what you see? if you set threshold=0.80, what does the model always predict? What is the implication for recall?
    4. Generate predicted classifications for two different thresholds (30% and 60% threshold). Save these new predictions as y_hat_30 and y_hat_60. Which threshold should you use if your goal is to avoid too many false negatives? Explain your answer. 
    5. Construct a data frame named df_predictions with 5 columns. y_test,  and the 4 y_hats from previous part

6. Borrow my_logistic_report() function from the python notebook of class 11. (25 points)
    1. Report the Accuracy, precision, recall and f1 score along with the confusion matrix for threshold =0.5. Interpret all these statistics. Do you trust the accuracy of the model? why? 
    2. Now use threshold = 0.3 in the my_logistic_report() function. what happens to accuracy, precision, recall and f1 score? what happens to false negatives? is this consistent with you answer to question 5.4? 

7. Plot the ROC curve and report the AUC score. Is your model doing a better job than random prediction (no skill)? 

8. Estimate the accuracy_test using K-Fold Cross Validation technique (try K=5 and K=10) and name them as accuracy_CV5 and accuracy_CV10. Are these numbers close to accuracy score from part 6.1? report your numbers with 5 digits precision. Why do you think all these 3 numbers are very close to each other? 